# Enter State Farm

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [1]:
%matplotlib inline
from __future__ import print_function, division
path = "data/state/"
#path = "data/state/sample/"
import imp
import utils
from utils import *
from IPython.display import FileLink

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size= 64

## Setup batches

In [3]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)
test_batches = get_batches(path+"test",batch_size=batch_size*2,shuffle=False)

Found 20924 images belonging to 10 classes.
Found 1500 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [4]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 20924 images belonging to 10 classes.
Found 1500 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


Rather than using batches, we could just import all the data into an array to save some processing time. (In most examples I'm using the batches, however - just because that's how I happened to start out.)

与其使用批处理，不如将所有数据导入到数组中，以节省一些处理时间。(不过，在大多数例子中，我使用批次——只是因为我恰好就是这样开始的。)

In [6]:
trn = get_data(path+'train')
val = get_data(path+'valid')

Found 20924 images belonging to 10 classes.
Found 1500 images belonging to 10 classes.


In [7]:
save_array(path+'results/val.dat', val)
save_array(path+'results/trn.dat', trn)

In [8]:
val = load_array(path+'results/val.dat')
trn = load_array(path+'results/trn.dat')

## Re-run sample experiments on full dataset

We should find that everything that worked on the sample (see statefarm-sample.ipynb), works on the full dataset too. Only better! Because now we have more data. So let's see how they go - the models in this section are exact copies of the sample notebook models.

### 单个卷积层

In [5]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Conv2D(filters=32,kernel_size=(3,3), activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Conv2D(filters=64,kernel_size=(3,3), activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    #model.fit_generator(batches, epochs=2, validation_data=val_batches)
    model.fit_generator(batches, steps_per_epoch=batches.n // batch_size, epochs=2, validation_data=val_batches, 
                    validation_steps=val_batches.n // batch_size)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, steps_per_epoch=batches.n // batch_size, epochs=4, validation_data=val_batches, 
                    validation_steps=val_batches.n // batch_size)
    return model

In [10]:
batches.n //batch_size

326

In [11]:
len(batches)

327

In [12]:
batches.n

20924

In [6]:
model = conv1(batches)

Epoch 1/2
326/326 [==============================] - 112s 342ms/step - loss: 0.2363 - acc: 0.9407 - val_loss: 0.0415 - val_acc: 0.9942
Epoch 2/2
326/326 [==============================] - 110s 339ms/step - loss: 0.0149 - acc: 0.9987 - val_loss: 0.0197 - val_acc: 0.9955
Epoch 1/4
326/326 [==============================] - 110s 338ms/step - loss: 0.0045 - acc: 0.9998 - val_loss: 0.0145 - val_acc: 0.9972
Epoch 2/4
326/326 [==============================] - 108s 332ms/step - loss: 0.0046 - acc: 0.9997 - val_loss: 0.0101 - val_acc: 0.9969
Epoch 3/4
326/326 [==============================] - 110s 336ms/step - loss: 0.0023 - acc: 0.9999 - val_loss: 0.0086 - val_acc: 0.9976
Epoch 4/4
326/326 [==============================] - 108s 331ms/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0074 - val_acc: 0.9972


In [9]:
model.save_weights(path+'models/single_conv.h5')

In [12]:
model.optimizer.lr = 1e-5

In [13]:
model.fit_generator(batches, steps_per_epoch=batches.n // batch_size, epochs=2, validation_data=val_batches, 
                    validation_steps=val_batches.n // batch_size)

Epoch 1/2
326/326 [==============================] - 125s 383ms/step - loss: 8.3423e-04 - acc: 1.0000 - val_loss: 0.0069 - val_acc: 0.9972
Epoch 2/2
326/326 [==============================] - 111s 340ms/step - loss: 6.8582e-04 - acc: 1.0000 - val_loss: 0.0061 - val_acc: 0.9979


In [14]:
model.fit_generator(batches, steps_per_epoch=batches.n // batch_size, epochs=5, validation_data=val_batches, 
                    validation_steps=val_batches.n // batch_size)

Epoch 1/5
326/326 [==============================] - 110s 338ms/step - loss: 5.1904e-04 - acc: 1.0000 - val_loss: 0.0056 - val_acc: 0.9979
Epoch 2/5
326/326 [==============================] - 110s 338ms/step - loss: 4.4860e-04 - acc: 1.0000 - val_loss: 0.0053 - val_acc: 0.9979
Epoch 3/5
326/326 [==============================] - 111s 340ms/step - loss: 3.6310e-04 - acc: 1.0000 - val_loss: 0.0048 - val_acc: 0.9979
Epoch 4/5
326/326 [==============================] - 112s 342ms/step - loss: 3.0750e-04 - acc: 1.0000 - val_loss: 0.0043 - val_acc: 0.9986
Epoch 5/5
326/326 [==============================] - 110s 337ms/step - loss: 2.6324e-04 - acc: 1.0000 - val_loss: 0.0044 - val_acc: 0.9979


Interestingly, with no regularization or augmentation we're getting some reasonable results from our simple convolutional model. So with augmentation, we hopefully will see some very good results.

有趣的是，由于没有正规化或增强，我们从简单的卷积模型得到了一些合理的结果。因此，通过增加，我们希望能看到一些很好的结果。

### 数据增广

In [37]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 20924 images belonging to 10 classes.


In [ ]:
model = conv1(batches)

In [ ]:
1e-4

In [ ]:
# 在之前的模型上，变更学习率为0.00001
model.optimizer.lr = 0.0001
# model.fit_generator(batches, steps_per_epoch=batches.n // batch_size, epochs=15, validation_data=val_batches, 
#                     validation_steps=val_batches.n // batch_size)
model.fit_generator(batches, epochs=15, validation_data=val_batches)

I'm shocked by *how* good these results are! We're regularly seeing 75-80% accuracy on the validation set, which puts us into the top third or better of the competition. With such a simple model and no dropout or semi-supervised learning, this really speaks to the power of this approach to data augmentation.

我对这些结果如此之好感到震惊!我们经常在验证集上看到75-80%的准确率，这使我们进入了竞争中排名前三或更高的位置。有了这样一个简单的模型，没有dropout或半监督学习，这就证明了这种方法对数据增强的作用。

### Four conv/pooling pairs + dropout 

Unfortunately, the results are still very unstable - the validation accuracy jumps from epoch to epoch. Perhaps a deeper model with some dropout would help.

由于显卡太烂，这个部分无法训练完成

In [41]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 20924 images belonging to 10 classes.


In [6]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Conv2D(32,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Conv2D(64,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Conv2D(128,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 3, 224, 224)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 222, 222)      896       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 222, 222)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 111, 111)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 109, 109)      18496     
_________________________________________________________________
batch_normalization_3 (Batch (None, 64, 109, 109)      256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 54, 54)        0         
__________

In [41]:
10e-5

0.0001

In [42]:
10e-4

0.001

In [8]:
model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit_generator(batches, steps_per_epoch=batches.n // batch_size, epochs=2, validation_data=val_batches, 
                    validation_steps=val_batches.n // batch_size)

Epoch 1/2
326/326 [==============================] - 356s 1s/step - loss: 2.4579 - acc: 0.2911 - val_loss: 0.8844 - val_acc: 0.7180
Epoch 2/2
326/326 [==============================] - 345s 1s/step - loss: 1.6295 - acc: 0.4888 - val_loss: 0.6182 - val_acc: 0.8119


In [10]:
model.optimizer.lr=0.001 # 提高学习率

In [11]:
model.fit_generator(batches, steps_per_epoch=batches.n // batch_size, epochs=10, validation_data=val_batches, 
                    validation_steps=val_batches.n // batch_size)
model.save_weights(path+'models/four_conv.h5')

Epoch 1/10
326/326 [==============================] - 339s 1s/step - loss: 1.1831 - acc: 0.6165 - val_loss: 0.3087 - val_acc: 0.9175
Epoch 2/10
326/326 [==============================] - 341s 1s/step - loss: 0.8898 - acc: 0.7039 - val_loss: 0.2103 - val_acc: 0.9415
Epoch 3/10
326/326 [==============================] - 336s 1s/step - loss: 0.7145 - acc: 0.7616 - val_loss: 0.1765 - val_acc: 0.9532
Epoch 4/10
326/326 [==============================] - 338s 1s/step - loss: 0.5823 - acc: 0.8082 - val_loss: 0.1408 - val_acc: 0.9587
Epoch 5/10
326/326 [==============================] - 344s 1s/step - loss: 0.4963 - acc: 0.8383 - val_loss: 0.1385 - val_acc: 0.9580
Epoch 6/10
326/326 [==============================] - 346s 1s/step - loss: 0.4112 - acc: 0.8643 - val_loss: 0.1169 - val_acc: 0.9721
Epoch 7/10
326/326 [==============================] - 349s 1s/step - loss: 0.3640 - acc: 0.8800 - val_loss: 0.0853 - val_acc: 0.9787
Epoch 8/10
326/326 [==============================] - 343s 1s/step - 

In [12]:
model.optimizer.lr=0.00001 # 降低学习率

In [13]:
model.fit_generator(batches, steps_per_epoch=batches.n // batch_size, epochs=10, validation_data=val_batches, 
                    validation_steps=val_batches.n // batch_size)

Epoch 1/10
326/326 [==============================] - 335s 1s/step - loss: 0.2340 - acc: 0.9277 - val_loss: 0.0740 - val_acc: 0.9790
Epoch 2/10
326/326 [==============================] - 335s 1s/step - loss: 0.2105 - acc: 0.9335 - val_loss: 0.0540 - val_acc: 0.9873
Epoch 3/10
326/326 [==============================] - 353s 1s/step - loss: 0.2030 - acc: 0.9353 - val_loss: 0.0592 - val_acc: 0.9842
Epoch 4/10
326/326 [==============================] - 331s 1s/step - loss: 0.1765 - acc: 0.9452 - val_loss: 0.0604 - val_acc: 0.9831
Epoch 5/10
326/326 [==============================] - 329s 1s/step - loss: 0.1845 - acc: 0.9432 - val_loss: 0.0542 - val_acc: 0.9862
Epoch 6/10
326/326 [==============================] - 332s 1s/step - loss: 0.1606 - acc: 0.9486 - val_loss: 0.0579 - val_acc: 0.9845
Epoch 7/10
326/326 [==============================] - 334s 1s/step - loss: 0.1514 - acc: 0.9517 - val_loss: 0.0541 - val_acc: 0.9818
Epoch 8/10
326/326 [==============================] - 331s 1s/step - 

In [14]:
model.save_weights(path+'models/four_conv.h5')

In [26]:
model.fit_generator(batches, steps_per_epoch=batches.n // batch_size, epochs=20, validation_data=val_batches, 
                    validation_steps=val_batches.n // batch_size)

Epoch 1/20
326/326 [==============================] - 360s 1s/step - loss: 0.1172 - acc: 0.9637 - val_loss: 0.0518 - val_acc: 0.9897
Epoch 2/20
326/326 [==============================] - 362s 1s/step - loss: 0.1198 - acc: 0.9643 - val_loss: 0.0495 - val_acc: 0.9880
Epoch 3/20
326/326 [==============================] - 364s 1s/step - loss: 0.1101 - acc: 0.9660 - val_loss: 0.0552 - val_acc: 0.9876
Epoch 4/20
326/326 [==============================] - 360s 1s/step - loss: 0.1081 - acc: 0.9661 - val_loss: 0.0687 - val_acc: 0.9790
Epoch 5/20
326/326 [==============================] - 358s 1s/step - loss: 0.1057 - acc: 0.9676 - val_loss: 0.0682 - val_acc: 0.9811
Epoch 6/20
326/326 [==============================] - 356s 1s/step - loss: 0.0980 - acc: 0.9679 - val_loss: 0.0466 - val_acc: 0.9869
Epoch 7/20
326/326 [==============================] - 353s 1s/step - loss: 0.0979 - acc: 0.9691 - val_loss: 0.0523 - val_acc: 0.9890
Epoch 8/20
326/326 [==============================] - 353s 1s/step - 

In [40]:
model.save_weights(path+'models/four_conv_20.h5')

This is looking quite a bit better - the accuracy is similar, but the stability is higher. There's still some way to go however...

### Imagenet conv features 有报错暂时无法尝试这里

Since we have so little data, and it is similar to imagenet images (full color photos), using pre-trained VGG weights is likely to be helpful - in fact it seems likely that we won't need to fine-tune the convolutional layer weights much, if at all. So we can pre-compute the output of the last convolutional layer, as we did in lesson 3 when we experimented with dropout. (However this means that we can't use full data augmentation, since we can't pre-compute something that changes every image.)


由于我们的数据如此之少，而且它与imagenet图像(全彩照片)相似，使用预先训练的VGG权重很可能是有用的——事实上，我们似乎不需要调整卷积层的权重，如果有的话。所以我们可以预先计算最后一个卷积层的输出，就像我们在第3课中做的那样，当我们做dropout实验的时候。(然而，这意味着我们不能使用全数据增强，因为我们不能预先计算改变每一张图像的东西。)

In [42]:
vgg = Vgg16()

In [43]:
model = vgg.model

In [44]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [45]:
# 寻找最后一个卷积层

for i,layer in enumerate(model.layers):
    if type(layer) is Conv2D:
        print(i,type(layer))

2 <class 'keras.layers.convolutional.Conv2D'>
4 <class 'keras.layers.convolutional.Conv2D'>
7 <class 'keras.layers.convolutional.Conv2D'>
9 <class 'keras.layers.convolutional.Conv2D'>
12 <class 'keras.layers.convolutional.Conv2D'>
14 <class 'keras.layers.convolutional.Conv2D'>
16 <class 'keras.layers.convolutional.Conv2D'>
19 <class 'keras.layers.convolutional.Conv2D'>
21 <class 'keras.layers.convolutional.Conv2D'>
23 <class 'keras.layers.convolutional.Conv2D'>
26 <class 'keras.layers.convolutional.Conv2D'>
28 <class 'keras.layers.convolutional.Conv2D'>
30 <class 'keras.layers.convolutional.Conv2D'>


In [46]:
[i for i,layer in enumerate(model.layers) if type(layer) is Conv2D]

[2, 4, 7, 9, 12, 14, 16, 19, 21, 23, 26, 28, 30]

In [47]:
last_conv_idx = [i for i,layer in enumerate(model.layers) if type(layer) is Conv2D][-1]

In [48]:
last_conv_idx

30

In [50]:
conv_layers = model.layers[:last_conv_idx+1]

In [52]:
# 建立顺序模型
conv_model = Sequential(conv_layers)

In [53]:
batch_size = 16

In [54]:
# batches shuffle must be set to False when pre-computing features
# 在预训练模型中，获取批量数据不能打乱
batches = get_batches(path+'train', batch_size=batch_size, shuffle=False)

Found 20924 images belonging to 10 classes.


In [55]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 20924 images belonging to 10 classes.
Found 1500 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [56]:
len(batches)

1308

In [57]:
conv_feat = conv_model.predict_generator(batches,verbose=1) # 训练

1145/1308 [=========================>....] - ETA: 13s

InternalError: Dst tensor is not initialized.
	 [[Node: conv2d_16_1/Relu/_677 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_162_conv2d_16_1/Relu", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:

conv_val_feat = conv_model.predict_generator(val_batches,steps=val_batches.n//batch_size) # 验证
conv_test_feat = conv_model.predict_generator(test_batches,steps=test_batches.n //batch_size) # 测试

In [ ]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_test_feat.dat', conv_test_feat)
save_array(path+'results/conv_feat.dat', conv_feat)

In [ ]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_val_feat.shape

### Batchnorm dense layers on pretrained conv layers 有问题

Since we've pre-computed the output of the last convolutional layer, we need to create a network that takes that as input, and predicts our 10 classes. Let's try using a simplified version of VGG's dense layers.

In [ ]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [ ]:
p=0.8

In [ ]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr=0.01

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=2, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.save_weights(path+'models/conv8.h5')

Looking good! Let's try pre-computing 5 epochs worth of augmented data, so we can experiment with combining dropout and augmentation on the pre-trained model.

### Pre-computed data augmentation + dropout

We'll use our usual data augmentation parameters:

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
da_batches = get_batches(path+'train', gen_t, batch_size=batch_size, shuffle=False)

We use those to create a dataset of convolutional features 5x bigger than the training set.

In [ ]:
da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample*5)

In [ ]:
save_array(path+'results/da_conv_feat2.dat', da_conv_feat)

In [ ]:
da_conv_feat = load_array(path+'results/da_conv_feat2.dat')

Let's include the real training data as well in its non-augmented form.

In [ ]:
da_conv_feat = np.concatenate([da_conv_feat, conv_feat])

Since we've now got a dataset 6x bigger than before, we'll need to copy our labels 6 times too.

In [ ]:
da_trn_labels = np.concatenate([trn_labels]*6)

Based on some experiments the previous model works well, with bigger dense layers.

In [ ]:
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [ ]:
p=0.8

In [ ]:
bn_model = Sequential(get_bn_da_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Now we can train the model as usual, with pre-computed augmented data.

In [ ]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr=0.01

In [ ]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr=0.0001

In [ ]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

Looks good - let's save those weights.

In [ ]:
bn_model.save_weights(path+'models/da_conv8_1.h5')

### Pseudo labeling

We're going to try using a combination of [pseudo labeling](http://deeplearning.net/wp-content/uploads/2013/03/pseudo_label_final.pdf) and [knowledge distillation](https://arxiv.org/abs/1503.02531) to allow us to use unlabeled data (i.e. do semi-supervised learning). For our initial experiment we'll use the validation set as the unlabeled data, so that we can see that it is working without using the test set. At a later date we'll try using the test set.

To do this, we simply calculate the predictions of our model...

In [ ]:
val_pseudo = bn_model.predict(conv_val_feat, batch_size=batch_size)

...concatenate them with our training labels...

In [ ]:
comb_pseudo = np.concatenate([da_trn_labels, val_pseudo])

In [ ]:
comb_feat = np.concatenate([da_conv_feat, conv_val_feat])

...and fine-tune our model using that data.

In [ ]:
bn_model.load_weights(path+'models/da_conv8_1.h5')

In [ ]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr=0.00001

In [ ]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

That's a distinct improvement - even although the validation set isn't very big. This looks encouraging for when we try this on the test set.

In [ ]:
bn_model.save_weights(path+'models/bn-ps8.h5')

### 提交

We'll find a good clipping amount using the validation set, prior to submitting.

In [27]:
def do_clip(arr, mx): 
    return np.clip(arr, (1-mx)/9, mx)

In [28]:
test_batches.n

79726

In [29]:
preds = model.predict_generator(test_batches,verbose=1)

623/623 [==============================] - 383s 614ms/step


In [30]:
preds.shape

(79726, 10)

In [31]:
subm = do_clip(preds,0.93)

In [32]:
subm_name = path+'results/subm.gz'

In [33]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)

In [34]:
classes

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [35]:
len(test_filenames)

79726

In [36]:
 [a[8:] for a in test_filenames]

['img_1.jpg',
 'img_10.jpg',
 'img_100.jpg',
 'img_1000.jpg',
 'img_100000.jpg',
 'img_100001.jpg',
 'img_100002.jpg',
 'img_100003.jpg',
 'img_100004.jpg',
 'img_100005.jpg',
 'img_100007.jpg',
 'img_100008.jpg',
 'img_100009.jpg',
 'img_10001.jpg',
 'img_100010.jpg',
 'img_100011.jpg',
 'img_100012.jpg',
 'img_100013.jpg',
 'img_100014.jpg',
 'img_100016.jpg',
 'img_100017.jpg',
 'img_100018.jpg',
 'img_100019.jpg',
 'img_10002.jpg',
 'img_100020.jpg',
 'img_100022.jpg',
 'img_100023.jpg',
 'img_100024.jpg',
 'img_100025.jpg',
 'img_100028.jpg',
 'img_100030.jpg',
 'img_100031.jpg',
 'img_100032.jpg',
 'img_100033.jpg',
 'img_100034.jpg',
 'img_100035.jpg',
 'img_100037.jpg',
 'img_100038.jpg',
 'img_100039.jpg',
 'img_10004.jpg',
 'img_100040.jpg',
 'img_100042.jpg',
 'img_100043.jpg',
 'img_100044.jpg',
 'img_100047.jpg',
 'img_100049.jpg',
 'img_10005.jpg',
 'img_100051.jpg',
 'img_100052.jpg',
 'img_100053.jpg',
 'img_100054.jpg',
 'img_100055.jpg',
 'img_100056.jpg',
 'img_10005

In [37]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'img', [a[8:] for a in test_filenames])
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_1.jpg,0.007778,0.007778,0.007778,0.007778,0.007778,0.930000,0.007778,0.007778,0.007778,0.007778
1,img_10.jpg,0.007778,0.007778,0.007778,0.007778,0.007778,0.930000,0.007778,0.007778,0.007778,0.007778
2,img_100.jpg,0.930000,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778
3,img_1000.jpg,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.930000,0.007778
4,img_100000.jpg,0.007778,0.007778,0.007778,0.930000,0.007778,0.007778,0.007778,0.007778,0.042645,0.007778


In [38]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [39]:
FileLink(subm_name)

D:\ML\courses\deeplearning1\nbs\data\state\results\subm.gz

This gets 0.534 on the leaderboard.